# Multinomial Classification using Netsaur

In [1]:
import {
  AdamOptimizer,
  Cost,
  CPU,
  DenseLayer,
  OneCycle,
  ReluLayer,
  Sequential,
  setupBackend,
  SoftmaxLayer,
  tensor,
  tensor1D,
  tensor2D,
} from "https://deno.land/x/netsaur@0.2.15/mod.ts";

import { parse } from "https://deno.land/std@0.204.0/csv/parse.ts";

// Import helpers for metrics
import {
  ClassificationReport,
  // Split the dataset
  useSplit,
  CategoricalEncoder,
} from "https://deno.land/x/vectorizer@v0.3.5/mod.ts";

We first read our dataset.

In [2]:
// Read the training dataset
const _data = Deno.readTextFileSync("../datasets/iris.csv");
const data = parse(_data);
data.shift()

// Get the predictors (x) and targets (y)
const x = data.map((fl) => fl.slice(0, 4).map(Number));
const y = data.map((fl) => fl[4]);

const encoder = new CategoricalEncoder()
const encodedY = encoder.fit(y).transform<"f64">(y, "f64")

[
  "sepal length",
  "sepal width",
  "petal length",
  "petal width",
  "class"
]

In [3]:
// Split the dataset for training and testing
const [[x_train, y_train], [x_test, y_test]] = useSplit(
  { ratio: [7, 3], shuffle: true },
  x,
  encodedY
);

In [4]:
// Setup the CPU backend for Netsaur
await setupBackend(CPU);

// Create a sequential neural network
const net = new Sequential({
  // Set number of minibatches to 6
  // Set size of input layer to 4
  size: [6, 4],

  // Disable logging during training
  silent: true,

  // Define each layer of the network
  layers: [
    // A dense layer with 16 neurons
    DenseLayer({ size: [16] }),
    // A ReLu activation layer
    ReluLayer(),
    // A dense layer with 3 neurons
    DenseLayer({ size: [3] }),
    // A Softmax activation layer
    SoftmaxLayer(),
  ],
  optimizer: AdamOptimizer(),
  // We are using CrossEntropy for finding cost
  cost: Cost.CrossEntropy,
  scheduler: OneCycle({ max_rate: 0.05, step_size: 50 }),
});


CPU Backend Initialized


In [5]:
const time = performance.now();
net.train(
  [
    {
      inputs: tensor2D(x_train),
      outputs: tensor(y_train.data, [y_train.nRows, y_train.nCols]),
    },
  ],
  300,
  1,
  0.02
);
console.log(`training time: ${performance.now() - time}ms`);

training time: 27.817799999999806ms


In [ ]:
// Calculate metrics
const res = await Promise.all(x_test.map(d => net.predict(tensor1D(d))));

const y1 = res.map((x) => encoder.getOg(x.data.indexOf(Math.max(...x.data))));
const y0 = encoder.untransform(y_test);
console.log(y1, y0)
const cReport = new ClassificationReport(y0, y1);
cReport;
